# Portfolio diversification<br>
<br>
Author: Sorin Moldoveanu<br>
Reference: all the credit goes to A. Simonetto et al; article published here: https://github.com/Qiskit/qiskit-tutorials/blob/master/qiskit/finance/optimization/portfolio_diversification.ipynb<br>
<br>
function to be minimized:<br>
$ min (H) = \sum_{i=1}^n \sum_{j=1}^n \rho_{ij} x_{ij} + A\Big( \sum_{j=1}^n y_j - q\Big)^2 + \sum_{i=1}^n A\Big( \sum_{j=1}^n x_{ij} - 1\Big)^2 + \sum_{j=1}^n A (x_{jj}-y_j)^2 +\sum_{i=1}^n \sum_{j=1}^n A \left(x_{ij} (1- y_j)\right)$<br>

where:<br>
$\rho_{ij}$ - similarity between stock i and stock j<br>
$x_{ij}, y_j $ - binary variables describing if stock j in in index fund (y) and which stock j is similar to i (x)<br>
$q$ - sotcks in fund<br>
$A$ - penalty coefficient<br>



In [1]:
#IMPORTS
import numpy as np
import datetime

import qiskit 
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua import Operator, run_algorithm
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.algorithms import VQE, QAOA, ExactEigensolver
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.components.variational_forms import RY

from qiskit.aqua.translators.data_providers import *
from qiskit.aqua.translators.ising import portfolio_diversification

#Data preparation
# Generate a pairwise time-series similarity matrix
stocks = ["TICKER1", "TICKER2"]
n = len(stocks)
rho = np.ones((n,n))
rho[0,1] = 0.8
rho[1,0] = 0.8

data = RandomDataProvider(tickers = stocks,
                 start = datetime.datetime(2016,1,1),
                 end = datetime.datetime(2016,1,30))
data.run()
rho = data.get_similarity_matrix()

# Actually, we consider the additive inverse to invert the direction of optimisation.  
rho = -1 * rho
#print('data:\n',data._data,'\nrho:\n',rho)

q=1 #number of clusters must be <= n

# Prepare processing
class QuantumOptimizer:

    def __init__(self, rho, n, q):

        self.rho = rho
        self.n = n
        self.q = q

    # Obtains the least eigenvalue of the Hamiltonian classically
    def exact_solution(self):
        qubitOp = portfolio_diversification.get_portfoliodiversification_qubitops(self.rho, self.n, self.q)
        algo_input = EnergyInput(qubitOp)
        algorithm_cfg = {
            'name': 'ExactEigensolver',
        }
        params = {
            'problem': {'name': 'ising'},
            'algorithm': algorithm_cfg
        }
        result = run_algorithm(params, algo_input)
        #print('\nresult:\n',result['eigvecs'][0])
        return self.decode_result(result)

    def vqe_solution(self):
        qubitOp = portfolio_diversification.get_portfoliodiversification_qubitops(self.rho, self.n, self.q)
        backend = BasicAer.get_backend('statevector_simulator')
        seed = 50
        cobyla = COBYLA()
        cobyla.set_options(maxiter=250)
        ry = RY(qubitOp.num_qubits, depth=5, entanglement='full')
        vqe = VQE(qubitOp, ry, cobyla, 'matrix')
        vqe.random_seed = seed
        quantum_instance = QuantumInstance(backend=backend, seed=seed, seed_transpiler=seed)
        result = vqe.run(quantum_instance)
        return self.decode_result(result)
 
    def decode_result(self, result, offset = 0):
        quantum_solution = portfolio_diversification.get_portfoliodiversification_solution(self.rho, self.n, self.q, result)
        ground_level = portfolio_diversification.get_portfoliodiversification_value(self.rho, self.n, self.q, quantum_solution)
        return quantum_solution, ground_level


In [2]:
# Instantiate the quantum optimizer class with parameters: 
quantum_optimizer = QuantumOptimizer(rho, n, q)
print('\nquant_opt:\n', quantum_optimizer.__dict__)


quant_opt:
 {'rho': array([[-1.       , -0.0019072],
       [-0.0019072, -1.       ]]), 'n': 2, 'q': 1}


In [3]:
#Exact solution
ground_state, ground_level = quantum_optimizer.exact_solution()
print('\nexact:\n',ground_state,'\nground level:\n',ground_level)


exact:
 [0 1 0 1 0 1] 
ground level:
 -1.0019071987512689


In [4]:
#VQE
vqe_state, vqe_level = quantum_optimizer.vqe_solution()
print('\nVQE:\n',vqe_state,'\nvqe level:\n',vqe_level)


VQE:
 [1 0 0 0 1 0] 
vqe level:
 999.0
